# Customer Lifetime Value Calculation

In [1]:
##########################################################
# CLTV = (Customer_Value / Churn_Rate) x Profit_margin.
##########################################################


import pandas as pd

In [2]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()

In [3]:
##################################################
# DATA PREPARATION
##################################################

In [4]:
# Invoice code starts with the letter 'C',indicates a cancellation.
# That's why chose the ones by using ~ which do not contains "C"
df = df[~df["Invoice"].astype(str).str.contains("C", na=False)]

# Be sure that each quantity value is bigger than 0:
df = df[(df['Quantity'] > 0)]

# Drop missing values:
df.dropna(inplace=True)

In [5]:
# TotalPrice variable is created by using Quantity and Price values:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [6]:
# cltv_df dataframe is created in order for ease of calculation and observation
cltv_df = df.groupby('Customer ID').agg({'Invoice': lambda x: len(x),
                                         'Quantity': lambda x: x.sum(),
                                         'TotalPrice': lambda x: x.sum()})

In [7]:
# cltv_df columns are renamed
# total transaction, total unit, total price
cltv_df.columns = ['total_transaction', 'total_unit', 'total_price']

In [8]:
cltv_df.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.0,1,74215,77183.60
12347.0,182,2458,4310.00
12348.0,31,2341,1797.24
12349.0,73,631,1757.55
12350.0,17,197,334.40


## Let's remember required formulas for Customer Lifetime Value Calculation

### - Average_Order_Value = Total_Revenue / Total_Number_of_Orders
### - Purchase_Frequency =  Total_Number_of_Orders / Total_Number_of_Customers
### - Customer_Value(CV)  = Average_Order_Value * Purchase_Frequency

### - Churn_Rate = 1 - Repeat_Rate
### - Profit_margin
### - CLTV = (Customer_Value / Churn_Rate) x Profit_margin.

In [9]:
# For CV calculation AOV and Frequency is required

#############################################################
# Average_Order_Value = Total_Revenue / Total_Number_of_Orders
#############################################################

cltv_df["avg_order_value"] = cltv_df["total_price"]/ cltv_df["total_transaction"]

In [10]:
##########################################################################
#Purchase Frequency= Total_Number_of_Orders / Total_Number_of_Customers
##########################################################################

#Customer number:
cltv_df.shape[0]

cltv_df["purchase_frequency"] = cltv_df["total_transaction"]/cltv_df.shape[0]

In [11]:
##################
# Repeat Rate & Churn Rate
##################

# If total transaction is bigger than this means that these customers were here more than once
# For repeat rate, the value where the total transaction is greater than 1 is selected.

#############################################################################################
# Repeat Rate = Number of customers who have purchased more than once / Number of Customers
#############################################################################################

repeat_rate = cltv_df[cltv_df.total_transaction > 1].shape[0]/cltv_df.shape[0]

# The churn rate, also known as the customer churn, is the rate at which customers stop doing business with an instution.
# Formula for the calculation of churn rate:
churn_rate = 1- repeat_rate

In [12]:
###############
#Profit Margin
################

# Here for this problem profit margin is chosen as 5%, actually the calculated value here can be called as Profit
# However it can be named as profit margin in order not to confuse the formulas.

# profit margin : 5 %
cltv_df["profit_margin"] = cltv_df["total_price"]* 0.05
cltv_df.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin
Customer ID,,,,,,
12346.0,1,74215,77183.60,77183.600000,0.000230,3859.1800
12347.0,182,2458,4310.00,23.681319,0.041945,215.5000
12348.0,31,2341,1797.24,57.975484,0.007145,89.8620
12349.0,73,631,1757.55,24.076027,0.016824,87.8775
12350.0,17,197,334.40,19.670588,0.003918,16.7200


In [13]:
##################
# Calculate Customer Lifetime Value
##################

# Customer Value(CV):
cltv_df["CV"] = cltv_df["avg_order_value"] * cltv_df["purchase_frequency"]
cltv_df.head()

#Customer Lifetime Value(CLTV):
cltv_df["CLTV"] = (cltv_df["CV"] * cltv_df["profit_margin"])/churn_rate

cltv_df.sort_values("CLTV", ascending = False)

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV
Customer ID,,,,,,,,
14646.0,2080,197491,280206.02,134.714433,0.479373,14010.3010,64.578479,5.452459e+07
18102.0,431,64124,259657.30,602.453132,0.099332,12982.8650,59.842660,4.682077e+07
17450.0,337,69993,194550.79,577.302047,0.077668,9727.5395,44.837702,2.628473e+07
16446.0,3,80997,168472.50,56157.500000,0.000691,8423.6250,38.827495,1.971041e+07
14911.0,5677,80515,143825.06,25.334694,1.308366,7191.2530,33.147052,1.436503e+07
...,...,...,...,...,...,...,...,...
17956.0,1,1,12.75,12.750000,0.000230,0.6375,0.002938,1.128906e-01
16454.0,2,3,6.90,3.450000,0.000461,0.3450,0.001590,3.306250e-02
14792.0,2,2,6.20,3.100000,0.000461,0.3100,0.001429,2.669444e-02


In [14]:
# Values are converted between 1-100 with minmaxscaler to better interpret the cltv value

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (1,100))
scaler.fit(cltv_df[["CLTV"]])
cltv_df["SCALED_CLTV"] = scaler.transform(cltv_df[["CLTV"]])
cltv_df.sort_values("CLTV", ascending = False)

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV,SCALED_CLTV
Customer ID,,,,,,,,,
14646.0,2080,197491,280206.02,134.714433,0.479373,14010.3010,64.578479,5.452459e+07,100.000000
18102.0,431,64124,259657.30,602.453132,0.099332,12982.8650,59.842660,4.682077e+07,86.012218
17450.0,337,69993,194550.79,577.302047,0.077668,9727.5395,44.837702,2.628473e+07,48.725036
16446.0,3,80997,168472.50,56157.500000,0.000691,8423.6250,38.827495,1.971041e+07,36.788073
14911.0,5677,80515,143825.06,25.334694,1.308366,7191.2530,33.147052,1.436503e+07,27.082511
...,...,...,...,...,...,...,...,...,...
17956.0,1,1,12.75,12.750000,0.000230,0.6375,0.002938,1.128906e-01,1.000000
16454.0,2,3,6.90,3.450000,0.000461,0.3450,0.001590,3.306250e-02,1.000000
14792.0,2,2,6.20,3.100000,0.000461,0.3100,0.001429,2.669444e-02,1.000000


In [15]:
# For comparison of values, variables are chosen from dataframe and values are sorted by "SCALED_CLTV"

cltv_df[["total_transaction", "total_unit","total_price","CLTV", "SCALED_CLTV"]].\
sort_values(by = "SCALED_CLTV",ascending= False).head()

,total_transaction,total_unit,total_price,CLTV,SCALED_CLTV
Customer ID,,,,,
14646.0,2080,197491,280206.02,5.452459e+07,100.000000
18102.0,431,64124,259657.30,4.682077e+07,86.012218
17450.0,337,69993,194550.79,2.628473e+07,48.725036
16446.0,3,80997,168472.50,1.971041e+07,36.788073
14911.0,5677,80515,143825.06,1.436503e+07,27.082511


In [16]:
# Segments are created by using qcut
# Segments are helpful to make the customer experience personalized

cltv_df["Segment"] = pd.qcut(cltv_df["SCALED_CLTV"], 4, labels = ["D", "C", "B", "A"])

In [17]:
# Total transaction, total unit, total price, CLTV and SCALED CLTV Values are interpreted with their count, sum, and mean values

cltv_df.groupby("Segment")[["total_transaction", "total_unit", "total_price", "CLTV", "SCALED_CLTV"]].agg(
    {"count", "mean", "sum"})

total_transaction                     total_unit           \
                    count     sum        mean      count      sum   
Segment                                                             
D                    1085   18682   17.218433       1085   140493   
C                    1085   40122   36.978802       1085   310632   
B                    1084   82856   76.435424       1084   703995   
A                    1085  256265  236.188940       1085  4026577   

                     total_price                            CLTV  \
                mean       count          sum         mean count   
Segment                                                            
D         129.486636        1085   194130.090   178.921742  1085   
C         286.296774        1085   503836.852   464.365762  1085   
B         649.441882        1084  1161913.311  1071.875748  1084   
A        3711.130876        1085  7051545.651  6499.120416  1085   

                                     SCALED_CLTV                         
                  sum           mean       count          sum      mean  
Segment                                                                  
D        2.810700e+04      25.905066        1085  1085.051034  1.000047  
C        1.716202e+05     158.175331        1085  1085.311610  1.000287  
B        9.247395e+05     853.080696        1084  1085.679044  1.001549  
A        2.549611e+08  234987.181486        1085  1547.931437  1.426665